In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
import pickle as pkl


cwd = os.getcwd()
model_path = os.path.abspath(os.path.join(cwd, os.pardir, 'models'))
sys.path.append(model_path)
from conference_model import *

In [ ]:
# Network
# Assuming 'SFHH_ConferenceNetwork.dat' is in the same directory and properly formatted
df = pd.read_csv('../data/SFHH_ConferenceNetwork.dat', delim_whitespace=True, header=None, names=['Timestamp', 'ID1', 'ID2'])

In [ ]:
# Parameters
s1 = 1/3.4
p_sev = 0.45
efficacy = 0.7
g_mild = 1/2.9
g_sev = 1/5.7
R0 = 2.8
t_interv = 0
t_end_conf = 5

prop_dict = {
    'Once daily': {'Mean': 0.78, 'SD': 0.14},
    'Twice daily': {'Mean': 0.69, 'SD': 0.15},
    '3 times a day': {'Mean': 0.64, 'SD': 0.15},
    '4 times a day': {'Mean': 0.51, 'SD': 0.20},
}

# Simulation setup
nr = 1000  # Number of realisations
Tmax = 50
dt = 0.01

In [ ]:
# Initial run for the untreated scenario (nr, dt, Tmax, A, s1, p_sev, g_mild, g_sev, efficacy, t_interv, R0, mean_prop, std_prop=None, t_end_conf=None, weighted='nonweighted', percentile=None):
_, _, IsimTot_untreated, cumul_mild_infections_untreated, cumul_sev_infections_untreated, cumul_infections_untreated, _, _, _, N = simulate_covid_spread(nr, dt, Tmax, df, s1, p_sev, g_mild, g_sev, 0, 0, R0, 0, t_end_conf=t_end_conf, weighted='weighted', percentile=90)



total_mild_infections_untreated = np.asarray([inner_array[-1] for inner_array in cumul_mild_infections_untreated])
total_sev_infections_untreated = np.asarray([inner_array[-1] for inner_array in cumul_sev_infections_untreated])

# Initialize a dictionary to store results
results = {}

# Loop through each scenario
for scenario, values in prop_dict.items():
    prop_mean = values['Mean']
    prop_std = values['SD']
    
    # Run simulation for the treated scenario
    _, _, IsimTot_treated, cumul_mild_infections_treated, cumul_sev_infections_treated, cumul_infections_treated, _, _, _, N = simulate_covid_spread(nr, dt, Tmax, df, s1, p_sev, g_mild, g_sev, efficacy, t_interv, R0, prop_mean, prop_std, t_end_conf=t_end_conf, weighted='weighted', percentile=90)


    # Calculate metrics for SEVERE infections and cumulative infections
    mean_infections = np.mean(IsimTot_treated, axis=0)
    lower_infections = np.percentile(IsimTot_treated, 2.5, axis=0)
    upper_infections = np.percentile(IsimTot_treated, 97.5, axis=0)
    mean_cumul_infections = np.mean(cumul_infections_treated, axis=0)
    lower_cumul_infections = np.percentile(cumul_infections_treated, 2.5, axis=0)
    upper_cumul_infections = np.percentile(cumul_infections_treated, 97.5, axis=0)
    
    # Calculate percentage of MILD infections avoided
    total_mild_infections_treated = np.asarray([inner_array[-1] for inner_array in cumul_mild_infections_treated])
    perc_mild_infections_avoided = 1 - total_mild_infections_treated / total_mild_infections_untreated
    mean_mild_averted = np.mean(perc_mild_infections_avoided)
    lower_mild_averted = np.percentile(perc_mild_infections_avoided, 2.5)
    upper_mild_averted = np.percentile(perc_mild_infections_avoided, 97.5)

    total_sev_infections_treated = np.asarray([inner_array[-1] for inner_array in cumul_sev_infections_treated])
    perc_sev_infections_avoided = 1 - total_sev_infections_treated / total_sev_infections_untreated
    mean_sev_averted = np.mean(perc_sev_infections_avoided)
    lower_sev_averted = np.percentile(perc_sev_infections_avoided, 2.5)
    upper_sev_averted = np.percentile(perc_sev_infections_avoided, 97.5)

    # Store results in the dictionary
    results[scenario] = {
        'Mean mild infections averted': mean_mild_averted,
        'Lower mild infections averted': lower_mild_averted,
        'Upper mild infections averted': upper_mild_averted,
        'Mean severe infections averted': mean_sev_averted,
        'Lower severe infections averted': lower_sev_averted,
        'Upper severe infections averted': upper_sev_averted,
        'Mean infections': mean_infections,
        'Lower infections': lower_infections,
        'Upper infections': upper_infections,
        'Mean cumul infections': mean_cumul_infections,
        'Lower cumul infections': lower_cumul_infections,
        'Upper cumul infections': upper_cumul_infections
    }

In [ ]:
# # Convert nested dictionary to DataFrame
# results_df = pd.DataFrame.from_dict(results, orient='index')

# # Save to CSV
# results_df.to_csv('20250616_conference_simulation_results.csv')

# Save both untreated and results dict
save_data = {
    'new_time_points': np.arange(0, Tmax, dt),
    'IsimTot_untreated': IsimTot_untreated,
    'cumul_infections_untreated': cumul_infections_untreated,
    't_end_conf': t_end_conf,
    'results': results
}

with open('conference_simulation_results.pkl', 'wb') as f:
    pkl.dump(save_data, f)

In [ ]:
# Initialize a dictionary to store results
results = {}

# Loop through each scenario
for scenario, values in prop_dict.items():
    prop_mean = values['Mean']
    prop_std = values['SD']
    
    # Run simulation for the treated scenario
    IsimMild_treated, IsimSev_treated, IsimTot_treated, cumul_mild_infections_treated, cumul_sev_infections_treated, cumul_infections_treated, cumul_unprotected_exposures, cumul_protected_exposures, props, N = simulate_covid_spread(nr, dt, Tmax, df, s1, p_sev, g_mild, g_sev, efficacy, t_interv, R0, prop_mean, prop_std, t_end_conf=t_end_conf, weighted='weighted', percentile=90)
    
    total_unprotected_exposures = np.asarray([inner_array[-1] for inner_array in cumul_unprotected_exposures])
    mean_unprotected_exposures = np.mean(total_unprotected_exposures)

    total_protected_exposures = np.asarray([inner_array[-1] for inner_array in cumul_protected_exposures])
    mean_protected_exposures = np.mean(total_protected_exposures)

    n_protected = props*N
    n_unprotected = (1-props)*N

    mean_n_protected = int(np.mean(n_protected))
    mean_n_unprotected = int(np.mean(n_unprotected))

    # Store results in the dictionary
    results[scenario] = {
        'Mean total unprotected': mean_n_unprotected,
        'Mean total protected': mean_n_protected,
        'Mean unprotected exposures': mean_unprotected_exposures,
        'Mean protected exposures': mean_protected_exposures
    }

results_df = pd.DataFrame.from_dict(results, orient='index').reset_index()
results_df.to_csv('../data/spray_nospray_results.csv', index=False)